In [29]:
import mysql.connector

conn = mysql.connector.connect(
    host="localhost",
    user="root",  
    password="password",  
    database="workshop_ml"
)


cursor = conn.cursor()

cursor.execute("""
CREATE TABLE IF NOT EXISTS predicciones (
    id INT AUTO_INCREMENT PRIMARY KEY,
    economy_gdp FLOAT,
    family FLOAT,
    health_life_expectancy FLOAT,
    freedom FLOAT,
    trust_government_corruption FLOAT,
    generosity FLOAT,
    happiness_score_predicho FLOAT
)
""")
print("Tabla 'predicciones' creada exitosamente (si no existía).")

cursor.close()
conn.close()


Tabla 'predicciones' creada exitosamente (si no existía).


In [30]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
import mysql.connector

df_combined = pd.read_csv('../Data/dataset_combinado.csv') 

X = df_combined[['Economy (GDP per Capita)', 'Family', 'Health (Life Expectancy)', 
                 'Freedom', 'Trust (Government Corruption)', 'Generosity']]
y = df_combined['Happiness Score']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

y_pred = rf_model.predict(X_test)

def guardar_predicciones_mysql(X_test, y_pred):
  
    conn = mysql.connector.connect(
        host="localhost",
        user="root",
        password="password",
        database="workshop_ml"
    )
    cursor = conn.cursor()

    for i in range(len(y_pred)):
        query = """
        INSERT INTO predicciones (economy_gdp, family, health_life_expectancy, freedom, trust_government_corruption, generosity, happiness_score_predicho)
        VALUES (%s, %s, %s, %s, %s, %s, %s)
        """
        
        values = (float(X_test[i][0]), float(X_test[i][1]), float(X_test[i][2]), 
                  float(X_test[i][3]), float(X_test[i][4]), float(X_test[i][5]), float(y_pred[i]))
        
        cursor.execute(query, values)

    
    conn.commit()
    print("Predicciones guardadas exitosamente en la base de datos.")

    cursor.close()
    conn.close()

guardar_predicciones_mysql(X_test.values, y_pred)


Predicciones guardadas exitosamente en la base de datos.
